In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout ,BatchNormalization, GlobalAveragePooling2D,MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from Evaluation_matrices import f1
import keras
from keras import layers , metrics 
from tensorflow.keras import layers, models
from keras.models import Model, load_model
from keras.callbacks import  ModelCheckpoint ,ReduceLROnPlateau ,EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.regularizers import l2
from Evaluation_matrices import f1
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import os
import cv2
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop, SGD


In [ ]:
data_dir = r'path to dataset'

In [ ]:
# Initialize empty lists to store images and labels
images = []
labels = []

# Loop through each subfolder (class) in the data directory
for class_name in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, class_name)
    # Ensure it's a directory
    if os.path.isdir(class_dir):
        # Loop through each image file in the subfolder
        for img_file in os.listdir(class_dir):
            # Read the image
            img_path = os.path.join(class_dir, img_file)
            # Check if image file exists
            if os.path.isfile(img_path):
                img = cv2.imread(img_path)
                # Check if image is loaded successfully
                if img is not None:
                    # Resize the image if necessary
                    img = cv2.resize(img, (224, 224))  # Adjust size as needed
                    # Append the image and corresponding label
                    images.append(img)
                    labels.append(class_name)
                else:
                    print(f"Error: Unable to load image '{img_path}'")
            else:
                print(f"Error: File '{img_path}' does not exist")

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Split the dataset into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)

In [ ]:
X_train = X_train/255
X_test = X_test/255
print(X_train.shape)
print(X_test.shape)
num_classes =y_train.shape[0]
print(num_classes)

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
print(len(y_train_encoded))

In [ ]:
model = Sequential()

model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

# Output layer with softmax activation for multi-class classification
model.add(Dense(len(np.unique(labels)), activation='softmax'))


In [ ]:
checkpoint = callbacks.ModelCheckpoint(r'path to save model weights', verbose=1, monitor='val_loss', save_best_only=True)

checkpoint1 = callbacks.ModelCheckpoint(r'path to save .h5',save_best_only=True, save_weights_only=True, verbose=1)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-10, verbose=1)

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

callbacks = [checkpoint1, checkpoint, reduce_lr,early_stop]




In [ ]:

opt_adam = tf.keras.optimizers.experimental.Adam(1e-3)
# Define RMSprop optimizer
#opt_rmsprop = RMSprop(learning_rate=1e-3)

# Define SGD optimizer with momentum
#opt_sgd = SGD(learning_rate=1e-3, momentum=0.9)
model.compile(loss='sparse_categorical_crossentropy',
              
              optimizer=opt_adam,
              metrics=['acc'])

model.summary()

In [ ]:
history = model.fit(train_generator, validation_data=validation_generator ,
          steps_per_epoch=40,
          epochs=120,
          shuffle=True, callbacks=callbacks)



In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Load your trained model
model = load_model(r'path to saved model')

# Initialize the video capture object
cap = cv2.VideoCapture(0)
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the captured frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the face region
        face_region = gray[y:y+h, x:x+w]

        # Preprocess the face region (resize, reshape, etc.) to match your model's input
        face_region_preprocessed = downsample_image(face_region)
        face_region_preprocessed = np.stack((face_region_preprocessed,)*3, axis=-1)  # Convert to 3 channels
        face_region_preprocessed = face_region_preprocessed.reshape(1, 224, 224, 3) / 255.0  # Add batch dimension and normalize

        # Predict the identity using your model
        predictions = model.predict(face_region_preprocessed)
        predicted_id  = np.argmax(predictions)
        predicted_identity = reverse_class_id_map[predicted_id] if predicted_id in reverse_class_id_map else "Unknown"

        # Display the predicted identity and ID on the frame
        display_text = f"ID: {predicted_id}, Name: {predicted_identity}"
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, display_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Frame', frame)

    # Break the loop with the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
